In [1]:
import tweepy
import json

In [2]:
API_Key = "i3BgiDWeRDGgx1MieH5pfsQdI"

API_Secret = "W3tQRuhqBhYCm2PSezofEb0N069DNOT2r36LZzdAOQW29FdWwy"

In [14]:
auth = tweepy.OAuthHandler(API_Key, API_Secret)

In [15]:
try:
    redirect_url = auth.get_authorization_url()
except tweepy.TweepError:
    print('Error! Failed to get request token.')

In [16]:
print(redirect_url)

https://api.twitter.com/oauth/authorize?oauth_token=2iG5HQAAAAABS6ayAAABe4TDN4k


In [17]:
pin = "1277511"

In [18]:
try:
    auth.get_access_token(pin)
except tweepy.TweepError:
    print('Error! Failed to get access token.')

In [19]:
print("token: " + auth.access_token )
print("secret: " + auth.access_token_secret)

token: 616379578-w3ElYEoF40uO4GyW0KoMAnzM0SG9lhiWhVsY0aj3
secret: nY86GsXMNmttcivMlYJ1iWnqFWhcBsWgLbFbdF5vln9Jz


In [ ]:
#save tokens


In [20]:
api = tweepy.API(auth)

In [58]:
#results = api.search_30_day("BootCamp","mn state fair")
results = api.search("#MnStateFair",count = 100)

In [59]:
dataset = []
for r in results:
    retweet = r._json["retweeted"]
    if not retweet:
        try:
            print(f" ++++++++++++ {r._json['extended_tweet']['full_text']}, {r._json['retweet_count']}, {r._json['favorite_count']}")
            tweet = r._json['extended_tweet']['full_text']
        except:
            print(f" ============ {r._json['text']}, {r._json['retweet_count']}, {r._json['favorite_count']}")
            tweet = r._json['text']
        dataset.append(tweet)
        
    else:
        print("--------------- retweeted ")
        
filename = "dataset3.txt"
textfile = open(filename, "w")
for element in dataset:
    textfile.write(element + "\n")
textfile.close()


 ============ RT @JayGabler: So on brand that the #mnstatefair crop art is protesting itself https://t.co/JAMn5oZj15, 141, 0
 ============ #mnstatefair me #boatlife https://t.co/Bq0Ywwdq4a, 0, 0
 ============ RT @JayGabler: So on brand that the #mnstatefair crop art is protesting itself https://t.co/JAMn5oZj15, 141, 0
 ============ RT @JodiLivon: On Sunday, August 29th at 11:30am I'm doing audience rapid readings at the Minnesota State Fair! Head to the #KSTP building…, 1, 0
 ============ RT @moricemdphd: Thanks @DanBarreiroKFAN @jgkfan for a one of a kind #mnstatefair experience! https://t.co/o5KEpTVDmT, 1, 0
 ============ RT @JayGabler: So on brand that the #mnstatefair crop art is protesting itself https://t.co/JAMn5oZj15, 141, 0
 ============ RT @JayGabler: So on brand that the #mnstatefair crop art is protesting itself https://t.co/JAMn5oZj15, 141, 0
 ============ RT @madisonstarrw: so pumped the #MNstatefair is open again this year 💖🌀🌺🦋🌀🌟 https://t.co/A9Kzhm8cGk, 1, 0
 ==========

In [61]:
results[0]._json

{'created_at': 'Fri Aug 27 01:49:06 +0000 2021',
 'id': 1431071239934709761,
 'id_str': '1431071239934709761',
 'text': 'RT @JayGabler: So on brand that the #mnstatefair crop art is protesting itself https://t.co/JAMn5oZj15',
 'truncated': False,
 'entities': {'hashtags': [{'text': 'mnstatefair', 'indices': [36, 48]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'JayGabler',
    'name': 'Jay Gabler',
    'id': 124558835,
    'id_str': '124558835',
    'indices': [3, 13]}],
  'urls': [],
  'media': [{'id': 1431031442386132995,
    'id_str': '1431031442386132995',
    'indices': [79, 102],
    'media_url': 'http://pbs.twimg.com/media/E9wLg9HXIAMULo4.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/E9wLg9HXIAMULo4.jpg',
    'url': 'https://t.co/JAMn5oZj15',
    'display_url': 'pic.twitter.com/JAMn5oZj15',
    'expanded_url': 'https://twitter.com/JayGabler/status/1431031454054723590/photo/1',
    'type': 'photo',
    'sizes': {'large': {'w': 2048, 'h': 1536, 'resize': 'fi

In [49]:
len(results)

100